In [2]:
!pip install transformers accelerate sentencepiece langdetect gradio wikipedia datasets faiss-cpu PyMuPDF torchaudio gtts tokenizer


In [3]:
# ---- IMPORTS ----
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from langdetect import detect
import gradio as gr
import wikipedia
import torch
import os
import fitz  # PyMuPDF
import faiss
from sklearn.feature_extraction.text import TfidfVectorizer
from gtts import gTTS
import requests
from bs4 import BeautifulSoup


In [4]:
# ---- TRANSLATION PIPELINES ----
# English to others
en_to_fr = pipeline("translation", model="Helsinki-Nlp/opus-mt-en-fr")
en_to_ar = pipeline("translation", model="Helsinki-Nlp/opus-mt-en-ar")
en_to_es = pipeline("translation", model="Helsinki-Nlp/opus-mt-en-es")

# Others to English
fr_to_en = pipeline("translation", model="Helsinki-Nlp/opus-mt-fr-en")
ar_to_en = pipeline("translation", model="Helsinki-Nlp/opus-mt-ar-en")
es_to_en = pipeline("translation", model="Helsinki-Nlp/opus-mt-es-en")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu
/usr/local/lib/python3.11/dist-packa

In [5]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

llm_tokenizer = AutoTokenizer.from_pretrained(model_name)
llm_model = AutoModelForCausalLM.from_pretrained(model_name)
llm_pipeline = pipeline("text-generation", model=llm_model, tokenizer=llm_tokenizer)
# ---- WHISPER FOR SPEECH INPUT ----
asr = pipeline("automatic-speech-recognition", model="openai/whisper-base")


Device set to use cpu
Device set to use cpu


In [7]:
# ---- TRANSLATION HELPERS ----
def translate_to_en(text):
    lang = detect(text)
    if lang == "fr":
        return fr_to_en(text)[0]['translation_text'], lang
    elif lang == "ar":
        return ar_to_en(text)[0]['translation_text'], lang
    elif lang == "es":
        return es_to_en(text)[0]['translation_text'], lang
    return text, "en"

def translate_from_en(text, target_lang):
    if target_lang == "fr":
        return en_to_fr(text)[0]['translation_text']
    elif target_lang == "ar":
        return en_to_ar(text)[0]['translation_text']
    elif target_lang == "es":
        return en_to_es(text)[0]['translation_text']
    return text

# ---- AUDIO ----
def transcribe_audio(file):
    return asr(file)["text"]

def synthesize_speech(text, lang_code="en"):
    tts = gTTS(text, lang=lang_code)
    file_path = "output.mp3"
    tts.save(file_path)
    return file_path

# ---- WIKIPEDIA RAG ----
def get_wikipedia_context(topic):
    try:
        page = wikipedia.page(topic)
        return page.content[:3000]
    except:
        return f"Could not retrieve content for '{topic}'. Try another topic."

def generate_final_answer(question, topic, lang):
    input_en, detected = translate_to_en(question)
    context = get_wikipedia_context(topic)

    prompt = f"""You are a helpful assistant. Use the following Wikipedia context to answer the question clearly and correctly.

Context:
{context}

Question:
{input_en}

Answer:"""
    raw_output = llm_pipeline(prompt, max_new_tokens=200)[0]['generated_text']
    answer = raw_output.split("Answer:")[-1].strip()
    return translate_from_en(answer, lang)

# ---- PDF MODE ----
def extract_text_from_pdf(file_path):
    with open(file_path, "rb") as f:
        pdf_bytes = f.read()
    doc = fitz.open(stream=pdf_bytes, filetype="pdf")
    return "\n".join(page.get_text() for page in doc)

def split_chunks(text, chunk_size=100, overlap=20):
    words = text.split()
    return [' '.join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size - overlap)]

def build_faiss_index(chunks):
    vectorizer = TfidfVectorizer().fit(chunks)
    vectors = vectorizer.transform(chunks).toarray().astype('float32')
    index = faiss.IndexFlatL2(vectors.shape[1])
    index.add(vectors)
    return index, vectorizer, chunks

def search_chunks(question, index, vectorizer, chunks, k=3):
    q_vec = vectorizer.transform([question]).toarray().astype('float32')
    _, indices = index.search(q_vec, k)
    return "\n".join([chunks[i] for i in indices[0] if i < len(chunks)])

def ask_from_pdf(pdf_file, question, lang):
    try:
        text = extract_text_from_pdf(pdf_file)
        chunks = split_chunks(text)
        index, vectorizer, chunks = build_faiss_index(chunks)
        relevant = search_chunks(question, index, vectorizer, chunks)

        translated_question, _ = translate_to_en(question)
        prompt = f"""You are a helpful assistant. Use the following document context to answer the question clearly.

Context:
{relevant}

Question:
{translated_question}

Answer:"""
        generated = llm_pipeline(prompt, max_new_tokens=200)[0]['generated_text']
        answer = generated.split("Answer:")[-1].strip()
        return translate_from_en(answer, lang)
    except Exception as e:
        return f"Error: {str(e)}"

        # ---- WEBSITE MODE ----

def get_text_from_url(url):
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, "html.parser")

        # Remove script and style tags
        for tag in soup(["script", "style"]):
            tag.extract()

        text = soup.get_text(separator=' ')
        return ' '.join(text.split())[:10000]  # Limit to 10k chars
    except Exception as e:
        return f"Failed to fetch content from {url}: {e}"

def ask_from_website(url, question, lang):
    website_text = get_text_from_url(url)
    chunks = split_chunks(website_text)
    index, vectorizer, chunk_list = build_faiss_index(chunks)
    relevant = search_chunks(question, index, vectorizer, chunk_list)

    translated_question, _ = translate_to_en(question)
    prompt = f"""You are a helpful assistant. Use the following website content to answer the question clearly.

Website Content:
{relevant}

Question:
{translated_question}

Answer:"""

    result = llm_pipeline(prompt, max_new_tokens=200)[0]['generated_text']
    answer = result.split("Answer:")[-1].strip()
    return translate_from_en(answer, lang)


In [8]:
# ---- INTERFACES ----
def chatbot_interface(question, topic, target_lang):
    return generate_final_answer(question, topic, target_lang)

def voice_to_voice_chat(audio_input, topic, target_lang):
    question_text = transcribe_audio(audio_input)
    answer_text = generate_final_answer(question_text, topic, target_lang)
    audio_output = synthesize_speech(answer_text, lang_code=target_lang)
    return question_text, answer_text, audio_output

# ---- GRADIO UI ----
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("##  Multilingual RAG Chatbot with TinyLlama")
    gr.Markdown("Ask your question by typing, speaking, or uploading a PDF.\\nSupports English, French, Arabic, and Spanish.")

    with gr.Tabs():
        with gr.TabItem("Text Mode"):
            topic = gr.Textbox(label="Topic")
            question = gr.Textbox(label="Your Question")
            lang = gr.Dropdown(["en", "fr", "ar", "es"], label="Output Language", value="en")
            output = gr.Markdown()
            gr.Button("Get Answer").click(fn=chatbot_interface, inputs=[question, topic, lang], outputs=output)

        with gr.TabItem("Voice Mode"):
            topic_voice = gr.Textbox(label="Topic")
            lang_voice = gr.Dropdown(["en", "fr", "ar", "es"], label="Output Language", value="en")
            audio_in = gr.Audio(type="filepath", label="Speak your question")
            transcript = gr.Textbox(label="Transcribed")
            answer = gr.Textbox(label="Answer")
            audio_out = gr.Audio(label="Spoken Answer")
            gr.Button("Get Spoken Answer").click(fn=voice_to_voice_chat, inputs=[audio_in, topic_voice, lang_voice],
                                                 outputs=[transcript, answer, audio_out])

        with gr.TabItem("Document Upload"):
            pdf_file = gr.File(label="Upload PDF", file_types=[".pdf"])
            pdf_question = gr.Textbox(label="Ask a question from the PDF")
            pdf_lang = gr.Dropdown(["en", "fr", "ar", "es"], value="en", label="Output Language")
            pdf_answer = gr.Textbox(label="Answer")
            gr.Button("Ask from PDF").click(fn=ask_from_pdf, inputs=[pdf_file, pdf_question, pdf_lang], outputs=pdf_answer)

        with gr.TabItem("Website Mode"):
            url_input = gr.Textbox(label="Website URL")
            web_question = gr.Textbox(label="Ask a question from the website")
            web_lang = gr.Dropdown(["en", "fr", "ar", "es"], value="en", label="Output Language")
            web_answer = gr.Textbox(label="Answer")
            gr.Button("Ask from Website").click(fn=ask_from_website,inputs=[url_input, web_question, web_lang],outputs=web_answer)


demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://320a71904efede6bcf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
